In [133]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import urllib3
import time
import datetime

In [134]:
website = 'https://www.wetter.de/deutschland/wetter-berlin-18228265.html'
website_15days = 'https://www.wetter.de/deutschland/wetter-berlin-18228265/wetterprognose.html'

In [135]:
url = 'https://www.wetter.de/deutschland/wetter-berlin-18228265.html'
http = urllib3.PoolManager()
soup = BeautifulSoup(http.request('GET',website_15days).data)

print(soup.prettify())

/home/eis/anaconda3/envs/bccn/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/eis/anaconda3/envs/bccn/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/eis/anaconda3/envs/bccn/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="de"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="de"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="de"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="de">
 <!--<![endif]-->
 <head>
  <!-- partials/site/html/head -->
  <script src="//cdn.optimizely.com/js/5078271027.js">
  </script>
  <!-- head title -->
  <title>
   Wetter Berlin 14 Tage +1 - 15 Tage Wettertrend - wetter.de
  </title>
  <!-- head meta -->
  <meta content="6593CA878189EEEF3510F38B05DE90A1" name="msvalidate.01"/>
  <meta content="Aktuelle Wettervorhersage für die kommenden 15 Tage in Berlin, Berlin. Bei Wetter.de finden Sie die Wettervorhersage stundengenau für heute und die nächsten 15 Tage. Alle Wetterdaten für Berlin, Berlin" name="description"/>
  <meta content="Wetter, Wetterbericht, Wettervorschau, Wettervorhersage, Wetter Berlin, Wetterbericht Berlin, Wetterübersicht Berlin, Wettervorsch

In [136]:
max_days_prediction = 15
id_list_dif_days = [str(datetime.date.today() + datetime.timedelta(days=i)).replace("-","") for i in range(15)]
for day in id_list_dif_days:
    some_forcast = soup.findAll('div', id="item-"+day)
    for forcast in some_forcast:
        date_ger = str(forcast.find('div', {'class':"text-day"}).text)
        date_num = day[-2:]+'-'+day[-4:-2]+'-'+day[:4]
        max_temp = forcast.find('span', {'class':"wt-color-temperature-max"}).text
        min_temp = forcast.find('span', {'class':"wt-color-temperature-min"}).text
    print(date_ger, date_num, max_temp, min_temp)


Di 08-05-2018 26° 9°
Mi 09-05-2018 26° 12°
Do 10-05-2018 26° 14°
Fr 11-05-2018 19° 12°
Sa 12-05-2018 22° 9°
So 13-05-2018 25° 12°
Mo 14-05-2018 22° 12°
Di 15-05-2018 20° 9°
Mi 16-05-2018 21° 10°
Do 17-05-2018 22° 9°
Fr 18-05-2018 22° 9°
Sa 19-05-2018 22° 9°
So 20-05-2018 22° 10°
Mo 21-05-2018 22° 10°
Di 22-05-2018 22° 10°
